<a href="https://colab.research.google.com/github/logikon-ai/cot-eval/blob/main/notebooks/CoT_Leaderboard_Results_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring the `/\/` Open CoT Leaderboard Results

## Setup

### Init

In [ ]:
%pip install -Uq datasets huggingface_hub seaborn GitPython

In [2]:
EVAL_DATASET = "logikon/cot-eval-results"
TRACES_DATASET = "logikon/cot-eval-traces"
LOCAL_DIR = "cot-eval-results"
LOCAL_CONFIGS = "cot-traces"
SUBSETS = ["base","cot","orig"]

In [3]:
import getpass
from huggingface_hub import HfApi, login

HF_TOKEN = getpass.getpass("HF Token:")
API = HfApi(token=HF_TOKEN)
login(HF_TOKEN, add_to_git_credential=True)

HF Token:··········
Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
#!rm -r cot-eval-results
#!rm -r cot-traces

### Load evaluation results

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id=EVAL_DATASET,
    revision="main",
    local_dir=LOCAL_DIR,
    repo_type="dataset",
    max_workers=60,
    token=HF_TOKEN
)


In [7]:
# get all models for which results are stored
import glob

models = []
for path in glob.glob(f"{LOCAL_DIR}/data/*/*", recursive=False):
  models.append(path.replace(f"{LOCAL_DIR}/data/",""))
models

['Deci/DeciLM-7B',
 'Deci/DeciLM-7B-instruct',
 'allenai/tulu-2-7b',
 'microsoft/phi-2',
 '01-ai/Yi-6B',
 'mistralai/Mistral-7B-Instruct-v0.2',
 'mistralai/Mistral-7B-v0.1',
 'Intel/neural-chat-7b-v3-1',
 'meta-llama/Llama-2-7b-hf']

In [8]:
import json

import pandas as pd

results = []

for model in models:
    for subset in SUBSETS:
        result_files = glob.glob(f"{LOCAL_DIR}/data/{model}/{subset}/**/*.json", recursive=True)
        for json_filepath in result_files:
            with open(json_filepath) as fp:
                data = json.load(fp)
            if "results" in data.keys():
                for k,v in data["results"].items():
                    record = v.copy()
                    record["model"] = model
                    record["subset"] = subset
                    results.append(record)

df_results = pd.DataFrame(results)
del results

In [9]:
def split_alias(alias: str) -> pd.Series:
    if alias[-5:]=="_base":
        alias = alias[:-5]
    elif alias[-4:]=="_cot":
        alias = alias[:-4]

    if "_" not in alias:
        task = alias
        config = ""
    else:
        config, task = alias.split("_")

    return pd.Series({"task": task, "config": config})

df_results = pd.concat([df_results, df_results.alias.apply(split_alias)], axis=1)

In [10]:
df_results.head()

,"acc,none","acc_stderr,none",alias,model,subset,task,config
0,0.371747,0.029520,veritatis-velit_lsat-rc_base,Deci/DeciLM-7B,base,lsat-rc,veritatis-velit
1,0.250980,0.019218,veritatis-velit_lsat-lr_base,Deci/DeciLM-7B,base,lsat-lr,veritatis-velit
2,0.221739,0.027451,veritatis-velit_lsat-ar_base,Deci/DeciLM-7B,base,lsat-ar,veritatis-velit
3,0.290735,0.018164,veritatis-velit_logiqa_base,Deci/DeciLM-7B,base,logiqa,veritatis-velit
4,0.367048,0.012161,veritatis-velit_logiqa2_base,Deci/DeciLM-7B,base,logiqa2,veritatis-velit


In [11]:
df_tmp1 = df_results[df_results.subset.eq("cot")].sort_values(by=["model","task","config"])
df_tmp2 = df_results[df_results.subset.eq("base")].sort_values(by=["model","task","config"])

df_tmp1.reset_index(inplace=True, drop=True)
df_tmp2.reset_index(inplace=True, drop=True)

assert df_tmp1.config.to_list() == df_tmp2.config.to_list()
assert df_tmp1.model.to_list() == df_tmp2.model.to_list()
assert df_tmp1.task.to_list() == df_tmp2.task.to_list()

df_cot = df_tmp1[["model","task","config"]].copy()
df_cot["acc_cot"] = df_tmp1["acc,none"]
df_cot["acc_base"] = df_tmp2["acc,none"]

df_cot["delta_abs"] = df_cot.acc_cot - df_cot.acc_base
df_cot["delta_rel"] = (df_cot.acc_cot - df_cot.acc_base)/df_cot.acc_base


In [12]:
df_cot.head()

,model,task,config,acc_cot,acc_base,delta_abs,delta_rel
0,01-ai/Yi-6B,logiqa,accusantium-inventore,0.327476,0.301917,0.025559,0.084656
1,01-ai/Yi-6B,logiqa,amet-ullam,0.292332,0.284345,0.007987,0.028090
2,01-ai/Yi-6B,logiqa,illum-eaque,0.292332,0.271565,0.020767,0.076471
3,01-ai/Yi-6B,logiqa,libero-exercitationem,0.321086,0.300319,0.020767,0.069149
4,01-ai/Yi-6B,logiqa,magnam-eius,0.290735,0.265176,0.025559,0.096386


In [13]:
df_cot.groupby("model").count()["task"]

model
01-ai/Yi-6B                           30
Deci/DeciLM-7B                        30
Deci/DeciLM-7B-instruct               30
Intel/neural-chat-7b-v3-1             30
allenai/tulu-2-7b                     30
meta-llama/Llama-2-7b-hf              30
microsoft/phi-2                       30
mistralai/Mistral-7B-Instruct-v0.2    30
mistralai/Mistral-7B-v0.1             30
Name: task, dtype: int64

### Load CoT configuration data

In [14]:
from git import Repo

repo = Repo.clone_from(f"https://huggingface.co/datasets/{TRACES_DATASET}", LOCAL_CONFIGS)

repo

<git.repo.base.Repo '/content/cot-traces/.git'>

In [15]:
COT_CONFIG_KEYS = ["cot_chain", "best_of", "use_beam_search", "temperature", "top_p", "top_k"]

def extract_config(message: str):
  config_dict = {}
  for line in message.split("\n"):
    line = line.strip()
    for key in COT_CONFIG_KEYS:
      if line.startswith(f"{key}:"):
        config_dict[key] = line.replace(f"{key}:","").strip()
  return config_dict



for c in repo.iter_commits(all=True, max_count=1, paths="ullam-consectetur-logiqa"):
  print(extract_config(c.message))

{'cot_chain': 'ReflectBeforeRun', 'best_of': '2', 'temperature': '0', 'top_k': '-1', 'top_p': '1.0', 'use_beam_search': 'true'}


In [16]:
cot_configs_data = {}

for cot_config_id in df_results.config.unique():
  if cot_config_id:
    c = next(repo.iter_commits(all=True, max_count=1, paths=f"{cot_config_id}-logiqa"))
    cot_configs_data[cot_config_id] = extract_config(c.message)


In [17]:
# add cot configs data to df_results
df_cot = pd.concat(
    [
        df_cot,
        df_cot.config.apply(lambda x: pd.Series(cot_configs_data[x]))
    ],
    axis=1
)


In [18]:
df_cot["decoding"] = "T:"+df_cot.temperature.astype(str) + "-BS:" + df_cot.use_beam_search.astype(str)

In [19]:
df_cot.head()

,model,task,config,acc_cot,acc_base,delta_abs,delta_rel,cot_chain,best_of,temperature,top_k,top_p,use_beam_search,decoding
0,01-ai/Yi-6B,logiqa,accusantium-inventore,0.327476,0.301917,0.025559,0.084656,HandsOn,1,0,-1,1.0,false,T:0-BS:false
1,01-ai/Yi-6B,logiqa,amet-ullam,0.292332,0.284345,0.007987,0.028090,ReflectBeforeRun,2,0,-1,1.0,true,T:0-BS:true
2,01-ai/Yi-6B,logiqa,illum-eaque,0.292332,0.271565,0.020767,0.076471,ReflectBeforeRun,1,0.3,100,0.95,false,T:0.3-BS:false
3,01-ai/Yi-6B,logiqa,libero-exercitationem,0.321086,0.300319,0.020767,0.069149,HandsOn,2,0,-1,1.0,true,T:0-BS:true
4,01-ai/Yi-6B,logiqa,magnam-eius,0.290735,0.265176,0.025559,0.096386,HandsOn,1,0.3,100,0.95,false,T:0.3-BS:false


## Exploration

#### CoT Accuracy Gains for Models and Tasks

(Averaged over all regimes.)

In [20]:
import seaborn as sns

sns.set_style("whitegrid")

sns.catplot(data=df_cot, hue="task", y="model", x="delta_abs", kind="bar", errorbar=("pi", 100))

#### CoT Accuracy Gains as a Function of Base Accuracy

In [23]:
sns.relplot(data=df_cot, col="task", hue="task", style="model", y="delta_abs", x="acc_base", kind="scatter")

#### CoT Accuracy Gains as a Function of Reasoning Regime

(Averaged over all models.)

In [24]:
sns.catplot(data=df_cot, hue="task", y="decoding", x="delta_abs", kind="bar", errorbar=("pi", 100), col="cot_chain")